In [3]:
# Obtenemos datos json con 25 elementos = vanos

import json

path = "../data/"

with open(path+'vanos.json', 'r') as archivo:
    data = json.load(archivo)
    
len(data)

25

In [4]:
# Mostramos las keys del primer elemento

print (len(data))

chiavi_primo_elemento = data[0].keys()

print (chiavi_primo_elemento)

25
dict_keys(['OBJECTID_VANO_2D', 'ID_VANO', 'LONGITUD_2D', 'COORDENADA_X_INICIO', 'COORDENADA_Y_INICIO', 'COORDENADA_X_FIN', 'COORDEANDA_Y_FIN', 'APOYOS', 'CONDUCTORES', 'LIDAR'])


In [5]:
# Desplegamos la estructura de datos para un elemento (listas, diccionaros, n elementos...)

element = data[3]

for key in element.keys():
    
    if type(element[key]) in [list, dict]:
        
        print(f"\n{key}: ")
        
        if type(element[key]) == list:
        
            element2 = element[key][0]
            print(f"- Length of list: {len(element[key])}")
            
        else:
            
            element2 = element[key]
        
        for key2 in element2.keys():
            
            print(f"    {key2}: {element2[key2]}")
            
            if type(element2[key2]) == list:
                print(f"    - Length of list: {len(element2[key2])}")
    
    else:
        print(f"\n{key}: {element[key]}")


OBJECTID_VANO_2D: 4

ID_VANO: C_13417187_13417204

LONGITUD_2D: 80.12791401818157

COORDENADA_X_INICIO: 382775.5

COORDENADA_Y_INICIO: 4460520.3719999995

COORDENADA_X_FIN: 382804.75100000016

COORDEANDA_Y_FIN: 4460445.774

APOYOS: 
- Length of list: 2
    OBJECTID_APOYO_3D: 19
    COD_APOYO: 13417204
    COORDENADA_X: 382775.5
    COORDEANDA_Y: 4460520.3719999995
    COORDENADAS_Z: [639.4060000000027, 648.2798999999941]
    - Length of list: 2

CONDUCTORES: 
- Length of list: 3
    OBJECTID_VANO_3D: 78
    LONGITUD_2D: 79.48033158625732
    LONGITUD_3D: 79.54355932137513
    VERTICES: [[382804.52749999985, 4460445.99, 650.8993000000046], [382801.2999999998, 4460454.2103, 650.3160000000062], [382798.07519999985, 4460462.431600001, 649.8031000000046], [382794.8530000001, 4460470.653999999, 649.3606], [382791.63350000046, 4460478.8773, 648.9882999999973], [382788.4166000001, 4460487.1017, 648.6861999999965], [382785.2022000002, 4460495.327099999, 648.4545000000071], [382781.99060000014,

In [6]:
#Número de puntos conductores:
numero_punti = len(data[0]['LIDAR']['CONDUCTORES'])
print("Numero di punti conductores:", numero_punti)

#Número de puntos de apoyo:
numero_punti = len(data[0]['LIDAR']['APOYOS'])
print("Numero di punti apoyos:", numero_punti)

Numero di punti conductores: 4612
Numero di punti apoyos: 1186


In [7]:
import plotly.graph_objects as go
import numpy as np

def get_coord(points):
    
    x_vals = [punto[0] for punto in points]
    y_vals = [punto[1] for punto in points]
    z_vals = [punto[2] for punto in points]
    
    return np.stack(x_vals), np.stack(y_vals), np.stack(z_vals)

def get_coord2(extremos_apoyos):
    
    x_vals = [extremos_apoyos[0]["COORDENADA_X"], extremos_apoyos[0]["COORDENADA_X"], extremos_apoyos[1]["COORDENADA_X"], extremos_apoyos[1]["COORDENADA_X"]]
    y_vals = [extremos_apoyos[0]["COORDEANDA_Y"], extremos_apoyos[0]["COORDEANDA_Y"], extremos_apoyos[1]["COORDEANDA_Y"], extremos_apoyos[1]["COORDEANDA_Y"]]
    z_vals = extremos_apoyos[0]["COORDENADAS_Z"] + extremos_apoyos[1]["COORDENADAS_Z"]

    return np.stack(x_vals), np.stack(y_vals), np.stack(z_vals)

In [8]:
puntos_conductores = data[0]['LIDAR']['CONDUCTORES']
puntos_apoyos = data[0]['LIDAR']['APOYOS']

extremos_apoyos = data[0]['APOYOS']
puntos_vertices = [element['VERTICES'][0] for element in data[0]['CONDUCTORES']]

x_vals_conductores, y_vals_conductores, z_vals_conductores = get_coord(puntos_conductores)
x_vals_apoyos, y_vals_apoyos, z_vals_apoyos = get_coord(puntos_apoyos)
x_vals_vert, y_vals_vert, z_vals_vert = get_coord(puntos_vertices)

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 3D Libs
import open3d as o3d
import laspy
print(laspy.__version__)

# Geospatial libs
import rasterio
import alphashape as ash
import geopandas as gpd
import shapely as sh    

from rasterio.transform import from_origin
from rasterio.enums import Resampling
from rasterio.features import shapes
from shapely.geometry import Polygon

2.5.3


### Data preparation

- Data curation
- Data Profiling
- Data Preprocess

In [16]:
# Define conductor coordinates

xyz_cond = np.vstack((x_vals_conductores, y_vals_conductores, z_vals_conductores))

# Transform to open3d and visualize

pcd_o3d = o3d.geometry.PointCloud()
pcd_o3d.points = o3d.utility.Vector3dVector(xyz_cond.transpose())

# Translate the point cloud and keep the translation for later
pcd_center = pcd_o3d.get_center()
pcd_o3d.translate(-pcd_center)

# Visualization
o3d.visualization.draw_geometries([pcd_o3d])

In [46]:
# Voxel conversion and visualizatio

voxels = pcd_o3d.voxel_down_sample(voxel_size=0.02)
o3d.visualization.draw_geometries([voxels])

In [74]:
# Point filtering based on NN and visualization

filtered_points = pcd_o3d.remove_statistical_outlier(nb_neighbors = 10, std_ratio = 1.0)

o3d.visualization.draw_geometries([filtered_points[0]])

In [75]:
# Point filtering based on radius and visualization

filtered_points = pcd_o3d.radius_outlier_removal(nb_points = 10, radius = 1.0)

o3d.visualization.draw_geometries([filtered_points[0]])

AttributeError: 'open3d.cuda.pybind.geometry.PointCloud' object has no attribute 'radius_outlier_removal'

In [68]:
# Get the average distance between points and deviance

nn_mean_distance = np.mean(pcd_o3d.compute_nearest_neighbor_distance())
nn_dev = np.std(pcd_o3d.compute_nearest_neighbor_distance())

print("Average distance to NN: ", nn_mean_distance, nn_dev)

# Substract points which NN distance is 1 sigmas from mean

idx = np.abs(pcd_o3d.compute_nearest_neighbor_distance() - nn_mean_distance) < 1*nn_dev

filtered_points = np.asarray(pcd_o3d.points)[idx]

filtered_o3d = o3d.geometry.PointCloud()
filtered_o3d.points = o3d.utility.Vector3dVector(filtered_points)

nn_mean_distance = np.mean(filtered_o3d.compute_nearest_neighbor_distance())
nn_dev = np.std(filtered_o3d.compute_nearest_neighbor_distance())

print("Average distance to NN: ", nn_mean_distance, nn_dev)

o3d.visualization.draw_geometries([filtered_o3d])

Average distance to NN:  0.053964314057631926 0.03056563258443211
Average distance to NN:  0.05176700806565719 0.01729873445065857


In [62]:
filtered_points

array([[ 0.08918701, -3.8769898 , -0.07511023],
       [ 0.06158701, -3.7327898 , -0.03361023],
       [ 0.08918701, -3.8769898 , -0.07511023],
       ...,
       [ 0.06158701, -3.7327898 , -0.03361023],
       [ 0.06158701, -3.7327898 , -0.03361023],
       [ 0.08918701, -3.8769898 , -0.07511023]])